In [205]:
import pandas as pd
from collections import Counter
import os
from urlextract import URLExtract 
import re
import numpy as np
from datetime import datetime
os.getcwd()

'/home/j0hndoe/Documents/git/reddit-disinformation/2_findflags/all_subreddits'

Generic functions

In [206]:
def has_url(text):
    return URLExtract().has_urls(str(text))
def CounterProc(x):
    c = Counter(x)
    return [{k:round(c[k]/sum([c[k] for k in c]),2)} for k in c]
def CounterProcOrdered(x):
    c = Counter(x).most_common()
    df = pd.DataFrame(c, columns = ['var','n'])
    df['proc'] = round(df['n'] / sum(df['n']),2)
    return df
def remove_spaces(txt):
    return(" ".join(str(txt).split()))


## Define classes for Subreddit, Submission, Comment

In [2]:
class reddit_subr(pd.DataFrame):
    
    @property
    def _constructor(self):
        return reddit_subr
    
    def selected_subreddits(self):
        return self.query('keep')
        
    def selected_columns(self):
        return self[['subreddit','category','subscribers_K','total7d']]
    
    def rename_subreddits(self):
        return self.rename(columns = {'subscribers_K':'sr_subscribers', 'category':'sr_category','total7d':'sr_ncovid'})

In [3]:
class reddit_subm(pd.DataFrame):
   
    @property
    def _constructor(self):
            return reddit_subm
    
    def new_names(self):
        return(self.rename(columns = {'id':'subm_id',
                             'created_utc':'subm_created',
                             'full_link':'subm_link',
                             'author':'subm_author',
                             'score':'subm_score',
                             'removed_by_category':'subm_removed',
                             'is_self':'subm_is_self',
                             'selftext':'subm_selftext',
                             'num_comments':'subm_ncomm',
                             'title':'subm_title',
                             'url':'subm_url',
                             'domain':'subm_domain',
                             'link_flair_text':'subm_flair'}))
    def new_order(self):
        return(self[['subreddit','subm_id','subm_title','subm_created',\
                  'subm_url','subm_domain','subm_author',\
                  'subm_score','subm_ncomm','subm_flair','subm_link']])

In [121]:
class reddit_comm(pd.DataFrame):
   
    @property
    def _constructor(self):
            return reddit_comm

    def new_names(self):
        return self.rename(columns = {'id':'comm_id',
                                     'created':'comm_created',
                                     'link_id':'subm_id',
                                     'parent_id':'comm_parent_id',
                                     #'body':'comm_body',
                                     'author':'comm_author',
                                     'score':'comm_score',
                                     'collapsed':'comm_collapsed'})
    
    def define_users(self):
        return self.assign(comm_author_class = lambda df: df['distinguished'].fillna('user')).\
                    assign(comm_body = lambda df: df['body'].astype(str))
        
    def new_order(self):
        return self[['subreddit','subm_id','comm_id',\
                      'comm_body', 'comm_created','comm_parent_id','comm_author',\
                      'comm_score','comm_collapsed','comm_author_class']]

## Read Subreddits, Submissions, Comments

For now, just reading April dataset

In [5]:
SR = pd.read_csv('../../input/subr_classification.csv')
SS = pd.read_csv('../../output/SUBM_2020_04.csv')
CC = pd.read_csv('../../output/COMM_2020_04.csv', lineterminator='\n')

### Setting conditions for keeping S & C

In [6]:
covid_regex = '|'.join(pd.read_csv('../../input/keywords_covid.txt',header=None).loc[:,0].\
                  str.replace("\\(|\\)",r"").str.replace(r"\*",r""))#r"\\w*"))
covid_regex

'corona|covid|virus|pandem|epidem|crisis|lockdown|quarantin|isolation|social distanc|deaths|death toll|infect|spread|patient|hospital|vaccin|symptom|ventilators|masks|medical suppl|cdc|world health organization|chloroquine|hydroxychlor'

In [7]:
condition_keep_subm = \
"not is_self and \
 not is_video and \
 not post_hint.fillna('dkna').str.contains('image|video|self') and \
 not domain.fillna('dkna').str.contains('reddit|redd.it|imgur|gfycat|google|twitter|youtu|^self\\.|^i\\.redd') and \
 num_comments>0"
condition_has_covid_regex = \
'(subreddit + title + full_link + selftext.fillna("")).str.contains("'+covid_regex+'")'

## Unification

### Define functions

In [8]:
def unite_subm_subr(SS,SR):
    return(\
        ### Read Submissions
        reddit_subm(SS).\
             ### Clean Submissions
             query(condition_keep_subm).\
                          query(condition_has_covid_regex).\
                          new_names().new_order().\
                          sort_values(['subreddit','subm_id','subm_created']).\
                          set_index(['subreddit','subm_id']).\
        ### Join Subreddits
        join(reddit_subr(SR).\
             ### Clean Subreddits
             selected_subreddits().\
             selected_columns().\
             rename_subreddits().\
             sort_values(['subreddit']).\
             set_index('subreddit'),
            how = 'left'))

In [9]:
def unite_all(CC,SS,SR):
    return(\
        ### Read Comments
        reddit_comm(CC).\
            ### Clean Comments
            new_names().define_users().new_order().\
            sort_values(['subreddit','subm_id','comm_created']).\
            set_index(['subreddit','subm_id','comm_id']).\
        ### Join Submissions
        join(reddit_subm(SS).\
             ### Clean Submissions
             query(condition_keep_subm).\
                          query(condition_has_covid_regex).\
                          new_names().new_order().\
                          sort_values(['subreddit','subm_id','subm_created']).\
                          set_index(['subreddit','subm_id']),
            how = 'inner').\
        ### Join Subreddits
        join(reddit_subr(SR).\
             ### Clean Subreddits
             selected_subreddits().\
             selected_columns().\
             rename_subreddits().\
             sort_values(['subreddit']).\
             set_index('subreddit'),
            how = 'left'))

### Unify

#### Submissions with Subreddits

In [10]:
SBSR = unite_subm_subr(SS,SR)
SBSR.head()

subm_title  \
subreddit subm_id                                                        
COVID19   t3_fsmcq8  Mercedes-AMG F1 team develops new CPAP for COV...   
          t3_fsmp8l    San Diego VA hospital vent system spreads virus   
          t3_fsmv3w  A herd of goats performs a riot across the tow...   
          t3_fsn86r  Microwave treatments destroy the infectivity o...   
          t3_fsndda  Chinese study found that there is no significa...   

                            subm_created  \
subreddit subm_id                          
COVID19   t3_fsmcq8  2020-04-01 00:01:10   
          t3_fsmp8l  2020-04-01 00:19:15   
          t3_fsmv3w  2020-04-01 00:27:51   
          t3_fsn86r  2020-04-01 00:47:42   
          t3_fsndda  2020-04-01 00:55:44   

                                                              subm_url  \
subreddit subm_id                                                        
COVID19   t3_fsmcq8  https://www.cnet.com/roadshow/news/mercedes-am...   
          t3_fsmp8l  https://inewsource.org/2020/03/30/malfunction-...   
          t3_fsmv3w  https://www.bigmache.com/2020/03/31/a-herd-of-...   
          t3_fsn86r          https://pubmed.ncbi.nlm.nih.gov/15223557/   
          t3_fsndda                http://html.rhhz.net/zhlxbx/028.htm   

                                 subm_domain      subm_author  subm_score  \
subreddit subm_id                                                           
COVID19   t3_fsmcq8                 cnet.com    redditigation           2   
          t3_fsmp8l           inewsource.org          Ninjurk           1   
          t3_fsmv3w             bigmache.com  dominiclheureux           1   
          t3_fsn86r  pubmed.ncbi.nlm.nih.gov       Timpoblete           4   
          t3_fsndda            html.rhhz.net             trlv           1   

                     subm_ncomm  \
subreddit subm_id                 
COVID19   t3_fsmcq8           1   
          t3_fsmp8l           2   
          t3_fsmv3w           1   
          t3_fsn86r           8   
          t3_fsndda           4   

                                                            subm_flair  \
subreddit subm_id                                                        
COVID19   t3_fsmcq8                                            General   
          t3_fsmp8l                                                NaN   
          t3_fsmv3w                                            General   
          t3_fsn86r  Rule 3: No editorialized title Removed - Weekl...   
          t3_fsndda                     Rule 3: No editorialized title   

                                                             subm_link  \
subreddit subm_id                                                        
COVID19   t3_fsmcq8  https://www.reddit.com/r/COVID19/comments/fsmc...   
          t3_fsmp8l  https://www.reddit.com/r/COVID19/comments/fsmp...   
          t3_fsmv3w  https://www.reddit.com/r/COVID19/comments/fsmv...   
          t3_fsn86r  https://www.reddit.com/r/COVID19/comments/fsn8...   
          t3_fsndda  https://www.reddit.com/r/COVID19/comments/fsnd...   

                     sr_category  sr_subscribers  sr_ncovid  
subreddit subm_id                                            
COVID19   t3_fsmcq8  coronavirus             182      104.3  
          t3_fsmp8l  coronavirus             182      104.3  
          t3_fsmv3w  coronavirus             182      104.3  
          t3_fsn86r  coronavirus             182      104.3  
          t3_fsndda  coronavirus             182      104.3

#### Comments with Submissions with Subreddits

In [122]:
COMSBSR = unite_all(CC,SS,SR)
COMSBSR.head()

comm_body  \
subreddit subm_id   comm_id                                                         
COVID19   t3_fsmcq8 t1_fm27vhg  CPAP machines are much safer than ventilators ...   
          t3_fsmp8l t1_fm2a6x4  The title of your post does not contain the so...   
          t3_fsmv3w t1_fm2akqf  bigmache.com is a news outlet. If possible, pl...   
          t3_fsn86r t1_fm2cps1                                          [removed]   
                    t1_fm2cxrb  Are you saying that the solution to out proble...   

                                       comm_created comm_parent_id  \
subreddit subm_id   comm_id                                          
COVID19   t3_fsmcq8 t1_fm27vhg  2020-03-31 22:03:29      t3_fsmcq8   
          t3_fsmp8l t1_fm2a6x4  2020-03-31 22:24:22      t3_fsmp8l   
          t3_fsmv3w t1_fm2akqf  2020-03-31 22:27:51      t3_fsmv3w   
          t3_fsn86r t1_fm2cps1  2020-03-31 22:47:42      t3_fsn86r   
                    t1_fm2cxrb  2020-03-31 22:49:47      t3_fsn86r   

                                     comm_author  comm_score  comm_collapsed  \
subreddit subm_id   comm_id                                                    
COVID19   t3_fsmcq8 t1_fm27vhg     redditigation           1           False   
          t3_fsmp8l t1_fm2a6x4  JenniferColeRhuk           1           False   
          t3_fsmv3w t1_fm2akqf     AutoModerator           1           False   
          t3_fsn86r t1_fm2cps1         [deleted]           1            True   
                    t1_fm2cxrb        bpmorris92           1           False   

                               comm_author_class  \
subreddit subm_id   comm_id                        
COVID19   t3_fsmcq8 t1_fm27vhg              user   
          t3_fsmp8l t1_fm2a6x4         moderator   
          t3_fsmv3w t1_fm2akqf         moderator   
          t3_fsn86r t1_fm2cps1              user   
                    t1_fm2cxrb              user   

                                                                       subm_title  \
subreddit subm_id   comm_id                                                         
COVID19   t3_fsmcq8 t1_fm27vhg  Mercedes-AMG F1 team develops new CPAP for COV...   
          t3_fsmp8l t1_fm2a6x4    San Diego VA hospital vent system spreads virus   
          t3_fsmv3w t1_fm2akqf  A herd of goats performs a riot across the tow...   
          t3_fsn86r t1_fm2cps1  Microwave treatments destroy the infectivity o...   
                    t1_fm2cxrb  Microwave treatments destroy the infectivity o...   

                                       subm_created  \
subreddit subm_id   comm_id                           
COVID19   t3_fsmcq8 t1_fm27vhg  2020-04-01 00:01:10   
          t3_fsmp8l t1_fm2a6x4  2020-04-01 00:19:15   
          t3_fsmv3w t1_fm2akqf  2020-04-01 00:27:51   
          t3_fsn86r t1_fm2cps1  2020-04-01 00:47:42   
                    t1_fm2cxrb  2020-04-01 00:47:42   

                                                                         subm_url  \
subreddit subm_id   comm_id                                                         
COVID19   t3_fsmcq8 t1_fm27vhg  https://www.cnet.com/roadshow/news/mercedes-am...   
          t3_fsmp8l t1_fm2a6x4  https://inewsource.org/2020/03/30/malfunction-...   
          t3_fsmv3w t1_fm2akqf  https://www.bigmache.com/2020/03/31/a-herd-of-...   
          t3_fsn86r t1_fm2cps1          https://pubmed.ncbi.nlm.nih.gov/15223557/   
                    t1_fm2cxrb          https://pubmed.ncbi.nlm.nih.gov/15223557/   

                                            subm_domain      subm_author  \
subreddit subm_id   comm_id                                                
COVID19   t3_fsmcq8 t1_fm27vhg                 cnet.com    redditigation   
          t3_fsmp8l t1_fm2a6x4           inewsource.org          Ninjurk   
          t3_fsmv3w t1_fm2akqf             bigmache.com  dominiclheureux   
          t3_fsn86r t1_fm2cps1  pubmed.ncbi.nlm.nih.gov       Timpoblete   
                    t1_fm

### Do not re-run these

In [327]:
%%timeit -r 3 -n 3 
COMSBSR = unite_all(CC,SS,SR)

16.2 s ± 119 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [219]:
SBSR.to_csv('../../output/SUBM_SUBR_'+str(datetime.now().date())+'.csv')

In [220]:
COMSBSR.to_csv('../../output/UNITED_'+str(datetime.now().date())+'.csv')

### Importing United datasets

We can import united dataset so we do not need to run again uniting syntax.

In [221]:
SBSR = pd.read_csv('../../output/SUBM_SUBR_2020-04-23.csv').set_index(['subreddit','subm_id'])
COMSBSR = pd.read_csv('../../output/UNITED_2020-04-23.csv').set_index(['subreddit','subm_id','comm_id'])
COMSBSR.shape

(1377155, 19)

In [222]:
COMSBSR.head()

comm_body  \
subreddit subm_id   comm_id                                                         
COVID19   t3_fsmcq8 t1_fm27vhg  CPAP machines are much safer than ventilators ...   
          t3_fsmp8l t1_fm2a6x4  The title of your post does not contain the so...   
          t3_fsmv3w t1_fm2akqf  bigmache.com is a news outlet. If possible, pl...   
          t3_fsn86r t1_fm2cps1                                          [removed]   
                    t1_fm2cxrb  Are you saying that the solution to out proble...   

                                       comm_created comm_parent_id  \
subreddit subm_id   comm_id                                          
COVID19   t3_fsmcq8 t1_fm27vhg  2020-03-31 22:03:29      t3_fsmcq8   
          t3_fsmp8l t1_fm2a6x4  2020-03-31 22:24:22      t3_fsmp8l   
          t3_fsmv3w t1_fm2akqf  2020-03-31 22:27:51      t3_fsmv3w   
          t3_fsn86r t1_fm2cps1  2020-03-31 22:47:42      t3_fsn86r   
                    t1_fm2cxrb  2020-03-31 22:49:47      t3_fsn86r   

                                     comm_author  comm_score  comm_collapsed  \
subreddit subm_id   comm_id                                                    
COVID19   t3_fsmcq8 t1_fm27vhg     redditigation           1           False   
          t3_fsmp8l t1_fm2a6x4  JenniferColeRhuk           1           False   
          t3_fsmv3w t1_fm2akqf     AutoModerator           1           False   
          t3_fsn86r t1_fm2cps1         [deleted]           1            True   
                    t1_fm2cxrb        bpmorris92           1           False   

                               comm_author_class  \
subreddit subm_id   comm_id                        
COVID19   t3_fsmcq8 t1_fm27vhg              user   
          t3_fsmp8l t1_fm2a6x4         moderator   
          t3_fsmv3w t1_fm2akqf         moderator   
          t3_fsn86r t1_fm2cps1              user   
                    t1_fm2cxrb              user   

                                                                       subm_title  \
subreddit subm_id   comm_id                                                         
COVID19   t3_fsmcq8 t1_fm27vhg  Mercedes-AMG F1 team develops new CPAP for COV...   
          t3_fsmp8l t1_fm2a6x4    San Diego VA hospital vent system spreads virus   
          t3_fsmv3w t1_fm2akqf  A herd of goats performs a riot across the tow...   
          t3_fsn86r t1_fm2cps1  Microwave treatments destroy the infectivity o...   
                    t1_fm2cxrb  Microwave treatments destroy the infectivity o...   

                                       subm_created  \
subreddit subm_id   comm_id                           
COVID19   t3_fsmcq8 t1_fm27vhg  2020-04-01 00:01:10   
          t3_fsmp8l t1_fm2a6x4  2020-04-01 00:19:15   
          t3_fsmv3w t1_fm2akqf  2020-04-01 00:27:51   
          t3_fsn86r t1_fm2cps1  2020-04-01 00:47:42   
                    t1_fm2cxrb  2020-04-01 00:47:42   

                                                                         subm_url  \
subreddit subm_id   comm_id                                                         
COVID19   t3_fsmcq8 t1_fm27vhg  https://www.cnet.com/roadshow/news/mercedes-am...   
          t3_fsmp8l t1_fm2a6x4  https://inewsource.org/2020/03/30/malfunction-...   
          t3_fsmv3w t1_fm2akqf  https://www.bigmache.com/2020/03/31/a-herd-of-...   
          t3_fsn86r t1_fm2cps1          https://pubmed.ncbi.nlm.nih.gov/15223557/   
                    t1_fm2cxrb          https://pubmed.ncbi.nlm.nih.gov/15223557/   

                                            subm_domain      subm_author  \
subreddit subm_id   comm_id                                                
COVID19   t3_fsmcq8 t1_fm27vhg                 cnet.com    redditigation   
          t3_fsmp8l t1_fm2a6x4           inewsource.org          Ninjurk   
          t3_fsmv3w t1_fm2akqf             bigmache.com  dominiclheureux   
          t3_fsn86r t1_fm2cps1  pubmed.ncbi.nlm.nih.gov       Timpoblete   
                    t1_fm

#### Codebook for United dataset

In [226]:
pd.read_csv('../../input/united_codebook.csv')

,VAR,LEVEL,INDEX,LABEL
0,subreddit,SUBREDDIT,yes,name of subreddit
1,subm_id,submission,yes,id of submission (t3_…..)
2,comm_id,comment,yes,id of comment (t1_…..)
3,comm_body,comment,NaN,text from comment
4,comm_created,comment,NaN,date and time comment was posted
5,comm_parent_id,comment,NaN,id of the comment this is a reply to (t3_… if ...
6,comm_author,comment,NaN,name of author (unique redditor)
7,comm_score,comment,NaN,score of comment (upvotes – downvotes)
8,comm_collapsed,comment,NaN,comment does not appear complete
9,comm_author_class,comment,NaN,author is also subreddit moderator?


### Analysis examples

Percent of submissions with score >1, by subreddit category

In [339]:
#SSQ.groupby('sr_category').agg({'subm_score': lambda x: round(np.exp(np.log(x+1).mean()),2)})
SBSR.groupby('sr_category').agg({'subm_score': lambda x: round((x>1).mean(),2)})

,subm_score
sr_category,
business_economics,0.36
coronavirus,0.38
generic_news,0.20
generic_politics,0.36
ideological,0.38
local,0.36
science_health,0.31
technology,0.34


Extract shared domains with higher percentage of removals

In [28]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(COMSBSR.assign(comm_del = lambda df: df['comm_body'] == '[removed]').\
                    groupby(['subreddit','subm_domain']).\
                    agg(count = ('comm_del', 'count'),
                        perc_removed = ('comm_del','mean')).\
                    reset_index().\
                    query('count>100').\
                    sort_values(['subreddit','perc_removed'], ascending = [True,False]).\
                    groupby(['subreddit']).head(3).
                    set_index(['subreddit','subm_domain']))

count  perc_removed
subreddit            subm_domain                                        
COVID19              thl.fi                            344      0.072674
                     sciencemag.org                    332      0.066265
                     covid19vaccinetrial.co.uk         129      0.062016
CanadaCoronavirus    theglobeandmail.com               175      0.005714
                     ctvnews.ca                        732      0.002732
                     cbc.ca                           1389      0.001440
CanadaPolitics       cbc.ca                            600      0.210000
                     theglobeandmail.com               709      0.208745
                     thestar.com                       201      0.203980
China                dailymail.co.uk                   141      0.063830
                     scmp.com                          231      0.038961
                     nypost.com                        171      0.035088
China_Flu            ctvnews.ca                        112      0.089286
                     ilmessaggero.it                   111      0.072072
                     abc.net.au                        123      0.056911
Conservative         ccn.com                           144      0.409722
                     babylonbee.com                    154      0.136364
                     pjmedia.com                       341      0.134897
Coronavirus          dnyuz.com                         419      0.188544
                     memorandumdaily.com              1000      0.125000
                     e.vnexpress.net                   412      0.109223
CoronavirusCA        thehill.com                       102      0.019608
                     mercurynews.com                   145      0.013793
                     latimes.com                       281      0.010676
CoronavirusDownunder smh.com.au                        202      0.089109
                     abc.net.au                        650      0.067692
                     theage.com.au                     226      0.066372
CoronavirusRecession marketwatch.com                   263      0.007605
                     wsws.org                          151      0.006623
                     cnbc.com                          323      0.000000
CoronavirusUK        express.co.uk                     115      0.086957
                     uk.reuters.com                    198      0.020202
                     metro.co.uk                       164      0.012195
CoronavirusUS        nypost.com                        162      0.049383
                     businessinsider.com               395      0.040506
                     vanityfair.com                    209      0.033493
CoronavirusWA        komonews.com                      140      0.000000
                     kuow.org                          364      0.000000
                     q13fox.com                        166      0.000000
DrainTheSwamp        foxnews.com                       207      0.024155
                     nypost.com                        348      0.014368
                     breitbart.com                     381      0.005249
Economics            thetimes.co.uk                    132      0.272727
                     bostonherald.com                  429      0.107226
                     marketwatch.com                   238      0.054622
EverythingScience    cnn.com                           241      0.074689
                     labroots.com                      153      0.058824
                     propublica.org                    144      0.048611
Futurology           theatlantic.com                   126      0.341270
                     goodnewznetwork.org               221      0.244344
                     cnn.com                          1041      0.240154
Health               nytimes.com                       126      0.103175
IndiaSpeaks          opindia.com                       616      0.029221
                     indiatoday.in    

## Fake News Regex

In [123]:
from IPython.core.display import display, HTML

def highlight_re(text, regex):
    htmld = re.sub(r'('+regex+')', '<mark>' + r'\1' + '</mark>', text.lower())
    display(HTML(htmld))
    
def print_submission(i, index, row):
    print('\n\n\nr/%s \n--------------------' % (index[0]))
    print('Published: %s' % row.subm_created[:20])
    print('%d | SUBM: %10s | COMM: %10s\n--------------------' % (i, index[1], index[2]))
    highlight_re('<b>'+row.subm_title+'</b>', covid_regex)
    print('URL: %s\nSubmission score: %d' % (row.subm_url, row.subm_score))

def print_comment(i, index, row, flag_no, limit = 1000):
    print('\nFLAG #%d (score = %d)' % (flag_no, row.comm_score))
    highlight_re(row.comm_body[:limit], fake_regex)    

In [124]:
fake_regex = '(fake |false).{0,15}(new\\w|info\\w|hood\\w)'+'|'+\
             '(dis|mis|mal)(info\\w|leading)'+'|'+\
             '\\bbs\\b|\\bbullshit\\b'+\
             '(not ).{0,5}(true\\b|verif\\w|real\\b)'+'|'+\
             '(unreli\\w+|unverifi\\w+|untrustw\\w+|fake\\w+)( source)'
fake_regex_no = '((no|not) .{0,10}('+fake_regex+'))'+'|(users often report submissions from this site)'

Extract comments with fake news regex

In [125]:
COMM_WITH_FLAGS = COMSBSR.query('comm_body.str.contains("'+fake_regex+'") and not comm_body.str.contains("'+fake_regex_no+'")')
COMM_WITH_FLAGS.shape

/home/j0hndoe/anaconda3/envs/jupyter-env/lib/python3.8/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


(10884, 19)

In [143]:
print('Percent of comments containing fake regex: %.2f' % (100*COMM_WITH_FLAGS.shape[0]/COMSBSR.shape[0])+'%')

Percent of comments containing fake regex: 0.79%


#### Select latest 1000 comments with COVID regex in submission and FAKE regex in comment

In [207]:
COMM_WITH_FLAGS_latest1000 = COMM_WITH_FLAGS.sort_values('subm_created',ascending=False)[:1000]

#### Print selection

In [201]:
current_submission_id = 'none'
for i,(index,row) in enumerate(COMM_WITH_FLAGS_latest1000.iterrows()):
    if current_submission_id==index[1]:
        flag_no += 1
    else:
        print_submission(i,index,row)
        flag_no = 1
    current_submission_id=index[1]
    print_comment(i,index,row,flag_no)




r/Coronavirus 
--------------------
Published: 2020-04-21 21:48:51
0 | SUBM:  t3_g5m85z | COMM: t1_fo45vzs
--------------------


URL: https://www.marketwatch.com/story/hydroxychloroquine-treatment-for-covid-19-shows-no-benefit-and-more-deaths-in-va-study-2020-04-21?mod=home-page
Submission score: 7

FLAG #1 (score = -2)





r/Coronavirus 
--------------------
Published: 2020-04-21 21:47:03
1 | SUBM:  t3_g5m6wx | COMM: t1_fo5n5mk
--------------------


URL: https://thehill.com/policy/healthcare/493931-largest-analysis-of-hydroxychloroquine-use-finds-no-benefit-increased
Submission score: 91

FLAG #1 (score = 1)



FLAG #2 (score = 0)


...




r/Coronavirus 
--------------------
Published: 2020-04-19 10:41:49
929 | SUBM:  t3_g44cvj | COMM: t1_fnw7ets
--------------------


URL: https://www.medrxiv.org/content/10.1101/2020.04.17.20053157v1
Submission score: 1

FLAG #1 (score = 1)



FLAG #2 (score = 1)





r/Futurology 
--------------------
Published: 2020-04-19 10:25:11
931 | SUBM:  t3_g446md | COMM: t1_fnv9dn4
--------------------


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

